In [21]:
import gdown
import os

url  = "https://drive.google.com/file/d/1Hb7Lgo5vtbAtr36fq6knylSiATlJRDS6/view?usp=share_link"
output = "./storage/jp2cache/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2"




storage_dir = "./storage"

if not os.path.exists(storage_dir + "/jp2cache"):
    os.makedirs(storage_dir + "/jp2cache")

In [2]:
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1Hb7Lgo5vtbAtr36fq6knylSiATlJRDS6
To: /workspace/trt_hovernet/storage/jp2cache/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2
100%|██████████████████████████████████████████████████████████████████████████████| 13.0G/13.0G [04:10<00:00, 52.0MB/s]


'./storage/jp2cache/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2'

In [3]:
!ls

'Multi-GPU Inference.ipynb'   hovernet_256_8_TF32.plan
 README.md		      hovernet_256_8_fp16.plan
 TensorRT		      hovernet_fast_pannuke_type_tf2pytorch.tar
 TensorRT_benchmarks.ipynb    jp2tileaccesor
 build.sh		      models
 dockerfile		      run_utils
 hovernet_256_16_TF32.plan    setup.sh
 hovernet_256_16_best.plan    setup_base
 hovernet_256_16_fp16.plan    storage


In [24]:
from jp2tileaccesor.multi_res_Tiling import SectionProxy, TileAccessor, Span
proxy_lossless = SectionProxy(15, "NISSL", 1708)

print(proxy_lossless)

{'id': 5944, 'position_index': 1708, 'jp2Path': '/data/storageIIT/humanbrain/analytics/37/NISL/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2', 'pngPathLow': '/data/storageIIT/humanbrain/analytics/37/NISL/B_37_FB3-SL_570-ST_NISL-SE_1708_thumbnail.jpg', 'width': 81590, 'height': 83590, 'series': 10, 'rigidrotation': 90, 'notes': None}{'brainid': 15, 'seriesType': 'NISSL', 'secnumber': 1708, 'modes': {'jp2': False, 'iip': True, 'mmap': False, 'tif': False}}


In [25]:
proxy_lossless.check_local_jp2()

./storage/jp2cache/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2


True

In [26]:
Image_size = 256
size = Span(Image_size,Image_size)
accessor = TileAccessor(proxy_lossless, 0, tilespan = size, use_iip = False)
print(accessor)

{'proxy': "{'id': 5944, 'position_index': 1708, 'jp2Path': '/data/storageIIT/humanbrain/analytics/37/NISL/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2', 'pngPathLow': '/data/storageIIT/humanbrain/analytics/37/NISL/B_37_FB3-SL_570-ST_NISL-SE_1708_thumbnail.jpg', 'width': 81590, 'height': 83590, 'series': 10, 'rigidrotation': 90, 'notes': None}{'brainid': 15, 'seriesType': 'NISSL', 'secnumber': 1708, 'modes': {'jp2': True, 'iip': True, 'mmap': False, 'tif': False}}", 'resolution': 0, 'imagespan': Span(w=81590, h=83590), 'tilespan': Span(w=256, h=256), 'ntiles': 104313, 'ntiles_c': 319, 'ntiles_r': 327}


In [29]:
import os
from PIL import Image
import numpy as np
import tensorrt as trt
import gc
import torch
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
num_tiles = 64
t=0
if not os.path.exists(storage_dir + "/cache_" + str(Image_size)):
    os.makedirs(storage_dir + "/cache_" + str(Image_size))

for i in range(0,accessor.ntiles):
    if t >= num_tiles:
        break
    tile = accessor[np.random.randint(accessor.ntiles)]
    if tile[0].shape == (256, 256, 3):
        plt.imsave(storage_dir+"/cache_"+str(Image_size) + "/"+str(i)+".png",tile[0])
        t+=1

In [27]:
file_list = os.listdir(storage_dir+"/cache_"+str(Image_size))
print("Number of Images",len(file_list))
pics = [ storage_dir + "/cache_"+str(Image_size) + "/" +str(i) for i in file_list]

Number of Images 64


In [30]:
np_pics = np.asarray(Image.open(pics[0]))
brain_tiles = np_pics.reshape(1,4,Image_size,Image_size)
for i in range(len(file_list)-1):
    brain_tiles = np.concatenate((brain_tiles,np.asarray(Image.open(pics[i+1])).reshape(1,4,Image_size,Image_size)),axis=0)
    

In [31]:
brain_tiles = torch.from_numpy(brain_tiles).float()

In [32]:
brain_tiles = brain_tiles[:,:3,:,:]

In [33]:
brain_tiles.shape

torch.Size([64, 3, 256, 256])

In [34]:
import TensorRT.inference as inf

In [35]:
TRT_LOGGER = trt.Logger(min_severity =trt.ILogger.INTERNAL_ERROR)
def load_engine(engine_file_path):
    assert os.path.exists(engine_file_path)
    print("Reading engine from file {}".format(engine_file_path))
    with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())


In [36]:
engine_files = {16 : ["hovernet_256_16_TF32.plan","hovernet_256_16_best.plan","hovernet_256_16_fp16.plan"], 8 : ["hovernet_256_8_TF32.plan","hovernet_256_8_fp16.plan"]}

In [37]:
from concurrent.futures import ThreadPoolExecutor

In [3]:
import ray

In [2]:
ray.init()

2023-01-16 11:08:11,085	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.2.0


(raylet) [2023-01-16 11:08:21,046 E 18917 18958] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-01-16_11-08-09_490333_18737 is over 95% full, available space: 0; capacity: 200512491520. Object creation will fail if spilling is required.
(raylet) [2023-01-16 11:08:31,050 E 18917 18958] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-01-16_11-08-09_490333_18737 is over 95% full, available space: 0; capacity: 200512491520. Object creation will fail if spilling is required.
(raylet) [2023-01-16 11:08:41,052 E 18917 18958] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-01-16_11-08-09_490333_18737 is over 95% full, available space: 0; capacity: 200512491520. Object creation will fail if spilling is required.
(raylet) [2023-01-16 11:08:51,055 E 18917 18958] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-01-16_11-08-09_490333_18737 is over 95% full, available space: 0; capacity: 200512491520. Object creation will fail if spilling is required.
(ray

In [38]:
import pycuda.driver as cuda
import pycuda.autoinit
import os

In [39]:
os.environ['CUDA_DEVICE'] = '0'

In [40]:

import nvidia_smi
nvidia_smi.nvmlInit()

In [41]:
nvidia_smi.nvmlUnitGetCount()

0

In [42]:
def run_inf(h_input_1, d_input_1, h_output, d_output, stream,images):
    inf.load_images_to_buffer(images, h_input_1)
    cuda.memcpy_htod_async(d_input_1, h_input_1, stream)
    context.execute_v2(bindings=[int(d_input_1), int(d_output)])
    cuda.memcpy_dtoh_async(h_output, d_output, stream)
    return h_output

In [48]:
%%capture
import tensorrt as trt
import pycuda.driver as cuda
import numpy as np
import pycuda.autoinit 
import nvidia_smi
nvidia_smi.nvmlInit()

results_run_engine = {}
results_time_in = {}
results_time_out = {}
results_time_load = {}
results_time_gpu_engine = {}
Batch_sizes = [8,16]

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

for j in Batch_sizes:
    batch_size = j
    for path_plan in engine_files[j]:
        
        res = []
        res2 = []
        res3 = []
        res4 = []
        res_gpu = []
        res_mem = []
        res_pwr = []
        engine = load_engine(path_plan)
        h_input_1, d_input_1, h_output, d_output, stream = inf.allocate_buffers(engine,1,trt.float32)
        with engine.create_execution_context() as context:
            for i in range(0,brain_tiles.shape[0],batch_size):
                temp = %timeit -n1 -r1 -o run_inf(h_input_1, d_input_1, h_output, d_output, stream,brain_tiles[i:i+batch_size,:,:,:])
                res.append(temp)
                g = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
                p = nvidia_smi.nvmlDeviceGetPowerUsage(handle)
                res_gpu.append(g.gpu)
                res_mem.append(g.memory)
                res_pwr.append(p/1000)

            a=0
            b=0
            w=0
            for i in res:
                a+=i.average
                b+=i.best
                w+=i.worst
            results_run_engine[path_plan] = (a,b,w)

            results_time_gpu_engine[path_plan] = (res_gpu,res_mem,res_pwr)

In [49]:
results_run_engine

{'hovernet_256_8_TF32.plan': (1.0334111978299916,
  1.0334111978299916,
  1.0334111978299916),
 'hovernet_256_8_fp16.plan': (0.43855243222787976,
  0.43855243222787976,
  0.43855243222787976),
 'hovernet_256_16_TF32.plan': (1.0294753164052963,
  1.0294753164052963,
  1.0294753164052963),
 'hovernet_256_16_best.plan': (0.288174559827894,
  0.288174559827894,
  0.288174559827894),
 'hovernet_256_16_fp16.plan': (0.41530073853209615,
  0.41530073853209615,
  0.41530073853209615)}